# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [73]:
import numpy as np 
import pandas as pd 
import xlsxwriter 
import math 
import requests
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [74]:
#grabbing list of S&P 500 Companies 
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
symbols_list = (df['Symbol'])
stocks = symbols_list.to_frame()

IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [75]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [76]:
price = data['latestPrice']
pe_ratio = data['peRatio']


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [77]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [57]:

simple_list = []

for symbol_string in symbol_strings: 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol == 'WLTW':
            pass
        else:
            simple_list.append([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['peRatio'], 'N/A'])



final_dataframe = pd.DataFrame(simple_list, columns=my_columns)






        

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [58]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]

final_dataframe.reset_index(inplace = True, drop = True)



## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [59]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [60]:
portfolio_input()


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [61]:
position_size = float(portfolio_size)/len(final_dataframe.index)
# position_size
# final_dataframe
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])




## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [62]:
# my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio' ,'Number of Shares to Buy']
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']

ev_to_ebitda = enterprise_value/ebitda
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [82]:
rv_columns = [
    'Symbol',
    'Price',
    'Number of Shares to Buy',
    'PE Ratio',
    'PE Percentile',
    'PB Ratio',
    'PB Percentile',
    'PS Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'PEG Ratio',
    'DE',
    'RV Score'

]


In [83]:
simple_list = []

for symbol_string in symbol_strings: 
    batch_api_call_url = batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        # if symbol == 'WLTW':
        #     pass
        # else:
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        
        except TypeError:
            ev_to_ebitda = np.NaN

        
        
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        simple_list.append([symbol, data[symbol]['quote']['latestPrice'], 'N/A', data[symbol]['quote']['peRatio'], 'N/A', data[symbol]['advanced-stats']['priceToBook'], 'N/A', data[symbol]['advanced-stats']['priceToSales'], 'N/A', ev_to_ebitda, 'N/A',ev_to_gross_profit, 'N/A',  data[symbol]['advanced-stats']['pegRatio'], data[symbol]['advanced-stats']['debtToEquity'],  'N/A'])


rv_dataframe = pd.DataFrame(simple_list, columns = rv_columns)









## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [65]:
rv_dataframe

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,182.708,N/A,17.96,N/A,7.19,N/A,3.05,N/A,12.604788,N/A,9.74064,N/A,N/A
1,AOS,87.300,N/A,30.00,N/A,7.20,N/A,4.00,N/A,19.238498,N/A,9.74064,N/A,N/A
2,ABT,136.050,N/A,34.49,N/A,7.28,N/A,5.70,N/A,23.755230,N/A,9.74064,N/A,N/A
3,ABBV,137.450,N/A,32.87,N/A,18.66,N/A,4.59,N/A,11.568818,N/A,9.74064,N/A,N/A
4,ABMD,323.610,N/A,99.38,N/A,10.74,N/A,15.30,N/A,47.807402,N/A,9.74064,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.690,N/A,25.90,N/A,-5.00,N/A,6.09,N/A,21.501106,N/A,9.74064,N/A,N/A
501,ZBRA,558.720,N/A,36.18,N/A,10.23,N/A,5.49,N/A,27.182246,N/A,9.74064,N/A,N/A
502,ZBH,127.200,N/A,33.63,N/A,2.19,N/A,3.50,N/A,18.083298,N/A,9.74064,N/A,N/A
503,ZION,70.726,N/A,6.89,N/A,1.50,N/A,3.24,N/A,5.153340,N/A,9.74064,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [66]:
# rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [67]:

exclude = []

# print(rv_columns[3:12:2])


# print(value)
for column in rv_columns[3:12:2]:

    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace= True)



## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [68]:
metrics = {}

for column in rv_dataframe.columns[3:12:2]:
    column1 = rv_dataframe.columns.get_loc(column)
    metrics.update({column : rv_dataframe.columns[column1 + 1] })

for metric in metrics: 
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])



## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [69]:
from statistics import mean



for row in rv_dataframe.index:
    pe = rv_dataframe.loc[row, 'PE Percentile']
    pb = rv_dataframe.loc[row, 'PB Percentile']
    ps = rv_dataframe.loc[row, 'PS Percentile']
    ev_ebitda = rv_dataframe.loc[row, 'EV/EBITDA Percentile']
    ev_gp = rv_dataframe.loc[row, 'EV/GP Percentile'] 
    rv_dataframe.loc[row, 'RV Score'] = mean([pe, pb, ps, ev_ebitda, ev_gp])



## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [70]:
best_value_stocks = rv_dataframe.sort_values(by = 'RV Score', ascending = True)
best_value_stocks = best_value_stocks[:50]
best_value_stocks.reset_index(drop = True, inplace = True )

best_value_stocks

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HPQ,39.990,N/A,3.78,6.930693,-13.0500,3.762376,0.3443,1.188119,4.221423,4.554455,9.74064,50.09901,13.306931
1,AAL,19.530,N/A,-3.86,5.544554,-1.6800,5.742574,0.5115,3.960396,-7.424757,1.980198,9.74064,50.09901,13.465347
2,TSN,94.110,N/A,5.54,8.316832,0.9453,8.712871,0.3526,1.386139,4.014380,3.960396,9.74064,50.09901,14.49505
3,WRK,46.899,N/A,7.52,10.891089,0.5366,5.940594,0.3290,0.990099,5.065262,5.742574,9.74064,50.09901,14.732673
4,PRU,116.560,N/A,6.39,8.910891,0.7262,6.534653,0.6377,4.554455,4.789240,5.346535,9.74064,50.09901,15.089109
5,DHI,99.330,N/A,4.44,7.524752,1.2300,10.49505,0.6486,4.950495,3.304839,2.970297,9.74064,50.09901,15.207921
6,LUMN,13.330,N/A,-18.91,3.762376,1.2600,10.891089,0.7143,6.534653,5.185256,6.336634,9.74064,50.09901,15.524752
7,HPE,17.800,N/A,3.46,6.336634,0.5549,6.138614,0.4065,2.178218,7.467485,14.257426,9.74064,50.09901,15.80198
8,BEN,36.933,N/A,5.10,7.722772,0.8436,7.524752,1.0900,12.178218,3.360167,3.168317,9.74064,50.09901,16.138614
9,MCK,258.170,N/A,-8.40,4.950495,-70.4800,1.980198,0.1570,0.594059,9.906609,26.336634,9.74064,50.09901,16.792079


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [71]:
portfolio_input()


In [72]:
for row in best_value_stocks.index:
    best_value_stocks.loc[row, 'Number of Shares to Buy'] = math.floor(float(portfolio_size)/best_value_stocks.loc[row, 'Price'])

best_value_stocks

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HPQ,39.990,250062,3.78,6.930693,-13.0500,3.762376,0.3443,1.188119,4.221423,4.554455,9.74064,50.09901,13.306931
1,AAL,19.530,512032,-3.86,5.544554,-1.6800,5.742574,0.5115,3.960396,-7.424757,1.980198,9.74064,50.09901,13.465347
2,TSN,94.110,106258,5.54,8.316832,0.9453,8.712871,0.3526,1.386139,4.014380,3.960396,9.74064,50.09901,14.49505
3,WRK,46.899,213224,7.52,10.891089,0.5366,5.940594,0.3290,0.990099,5.065262,5.742574,9.74064,50.09901,14.732673
4,PRU,116.560,85792,6.39,8.910891,0.7262,6.534653,0.6377,4.554455,4.789240,5.346535,9.74064,50.09901,15.089109
5,DHI,99.330,100674,4.44,7.524752,1.2300,10.49505,0.6486,4.950495,3.304839,2.970297,9.74064,50.09901,15.207921
6,LUMN,13.330,750187,-18.91,3.762376,1.2600,10.891089,0.7143,6.534653,5.185256,6.336634,9.74064,50.09901,15.524752
7,HPE,17.800,561797,3.46,6.336634,0.5549,6.138614,0.4065,2.178218,7.467485,14.257426,9.74064,50.09901,15.80198
8,BEN,36.933,270760,5.10,7.722772,0.8436,7.524752,1.0900,12.178218,3.360167,3.168317,9.74064,50.09901,16.138614
9,MCK,258.170,38734,-8.40,4.950495,-70.4800,1.980198,0.1570,0.594059,9.906609,26.336634,9.74064,50.09901,16.792079


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: